### Extract big "grouped" file of users with candidate sentiment or topic, from neo4j

In [ ]:
hourly_query="""
match (t:Tweet_{0})<-[:POSTS]-(u:User_{0}) \
where exists(t.mentions_Trump) \
or exists(t.mentions_Clinton) \
or exists(t.mentions_Sanders) \
or exists(t.mentions_Cruz) \
or exists(t.mentions_Rubio) \
or exists(t.topic) \
WITH t.mentions_Trump as Trump, \
t.mentions_Clinton as Clinton, \
t.mentions_Sanders as Sanders, \
t.mentions_Cruz as Cruz, \
t.mentions_Rubio as Rubio, \
t.topic as topic, \
u.screen_name as user, (toInt(t.timestamp_int) / 3600000) as hour_group, count(t) as frequency, \
MIN(t.timestamp_int) as mints \
RETURN user, mints, Trump, Clinton, Sanders, Cruz, Rubio, \
topic, frequency \
order by frequency desc"""

hourly_header=['year', 'month', 'day', 'hour', 'user', 'mints',\
               'Trump','Clinton','Sanders','Cruz','Rubio',\
               'topic','frequency']




In [ ]:
partitions = ['2015_10_11', '2015_10_12', '2015_10_13', '2015_10_14', '2015_10_15',\
              '2016_1_26', '2016_1_27', '2016_1_28', '2016_1_29', '2016_1_30', '2016_2_2', '2016_2_3', \
              '2016_2_4', '2016_2_5', '2016_2_6', '2016_2_7', '2016_2_8', '2016_2_9', '2016_2_10',\
              '2016_2_11', '2016_2_12', '2016_2_13', '2016_2_14', '2016_2_15', \
              '2016_2_23', '2016_2_24', '2016_2_25', '2016_2_26', '2016_2_27', \
              '2016_3_1', '2016_3_2','2016_3_3', '2016_3_4', '2016_3_5', '2016_3_6', '2016_3_7', \
              '2016_3_8', '2016_3_9', '2016_3_10', '2016_3_11']


In [ ]:
from datetime import datetime

# Append the day and hour adjusted to EST (UTC-5)
# convert ms to s for python
def converttime(ts):
    ts_adjust = (ts/1000) - (3600*5)
    dt = datetime.utcfromtimestamp(ts_adjust)
    return dt.year, dt.month, dt.day, dt.hour

# When an item is retweeted, we keep a copy of the original tweet (based on retweet_status).
# This copy might be from prior to our day.  we need to remove this data from the final result.
def keepRetweetedItem(partition, ts):
    ts_adjust = ts / 1000
    dt = datetime.utcfromtimestamp(ts_adjust)
    year, month, day = partition.split('_')
    return ((int(year)==dt.year) and (int(month)==dt.month) and (int(day)==dt.day))

print converttime(1444607999000)
print keepRetweetedItem('2015_10_11',1444607999000)
print keepRetweetedItem('2015_10_11',1428188930000)



In [ ]:
import time
import logging
import csv
from neo4jreader import neo4j_reader

#set up logging
logger = logging.getLogger('ResultsGroupedByCandidateTopic2')
hdlr = logging.FileHandler('./user_results2_tagging.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.INFO)

reader = neo4j_reader('local',logger)

with open('hourly_user_results2.csv', 'wb') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(hourly_header)
    
    #throwaway=0
    
    for partition in partitions:

        start = time.time()

        formatted_query = hourly_query.format(partition)
        #print formatted_query

        for record in reader.runquery(formatted_query):
            outrecord = []
            for i in range(len(record)):
                outrecord.append(record[i])

            mints = int(outrecord[1])
            
            #if (keepRetweetedItem(partition, mints)):
                
            #some of the neutral items were labeled true instead of neu, correct these.
            #TODO: correct these in the database.
            for n in range(2,7):
                if ("true" in str(outrecord[n]).lower()):
                    outrecord[n]="neu"

            #convert time from UTC to EST and add day/hour information to output
            year, month, day, hour = converttime(mints)
            outrecord = [year, month, day, hour] + outrecord

            csvwriter.writerow(outrecord)  
                
#             else:
#                 throwaway = throwaway + 1


        print partition, time.time() - start
        
    #print throwaway


        

### Convert Hourly to Daily

In [ ]:
import pandas as pd

df2 = pd.read_csv('hourly_user_results2.csv')



In [ ]:
df3a = df2[['year', 'month', 'day','user','Trump','Clinton','Sanders','Cruz','Rubio',\
               'topic','frequency']]



In [ ]:
df3 = df3a.reset_index()
df3.fillna('none',inplace=True)
df3

In [ ]:
#Daily data

df4 = df3.groupby([df3['year'],df3['month'],df3['day'],df3['user'],df3['Trump'],\
                   df3['Clinton'],df3['Sanders'],\
            df3['Cruz'],df3['Rubio'],df3['topic']]).sum().reset_index()
df4

In [ ]:
df4.to_csv('daily_user_results2.csv')

### Extract, pivot data, and merge back to create our transformed file.

In [1]:
import pandas as pd
df4a = pd.read_csv('daily_user_results2.csv')

In [2]:
import numpy as np

#use partition list to define list of days...process one day at a time and write output
#for partition in partitions:
    
#df4b = df4a[(df4a['year']==2015) & (df4a['month']==10) &(df4a['day']==15)]
df4c = df4a.replace('none', np.nan)



In [8]:
pivoted = pd.DataFrame()

lastcandidate=''

for candidate in ['Trump','Clinton','Sanders','Rubio','Cruz']:
    
    dfp = pd.pivot_table(df4c,index=["year","month","day","user"],values=["frequency"],
               columns=[candidate],aggfunc=[np.sum],fill_value=0)
    
    dfp.columns = dfp.columns.get_level_values(2)
    dfp.reset_index(inplace=True)
    dfp.columns = ['year','month','day','user', '_'.join([candidate,'neg']),'_'.join([candidate,'neu']),\
                   '_'.join([candidate,'pos'])]
    
    print dfp[0:5]
    
    if (lastcandidate==''):
        pivoted = dfp
    else:
        df_merged = pd.merge(pivoted, dfp, on=['year','month','day','user'],how='outer')
        df_merged.sort(['year','month','day','user'],inplace=True)
        df_merged[0:20]
        pivoted = df_merged
        
    lastcandidate = candidate
    print pivoted[0:5]
    

   year  month  day         user  Trump_neg  Trump_neu  Trump_pos
0  2006      8   31      kmikeym          0          1          0
1  2006     12   22      so_n_so          0          7          0
2  2007      5   12     ursonate          1          0          0
3  2007      8   24   mtvmoonman          0          1          0
4  2008      8   29  huffpostpol          1          0          0
   year  month  day         user  Trump_neg  Trump_neu  Trump_pos
0  2006      8   31      kmikeym          0          1          0
1  2006     12   22      so_n_so          0          7          0
2  2007      5   12     ursonate          1          0          0
3  2007      8   24   mtvmoonman          0          1          0
4  2008      8   29  huffpostpol          1          0          0
   year  month  day       user  Clinton_neg  Clinton_neu  Clinton_pos
0  2008      1    9  market_jp            0            1            0
1  2008      1   10   rnz_news            0            1            

In [9]:
pivoted

,year,month,day,user,Trump_neg,Trump_neu,Trump_pos,Clinton_neg,Clinton_neu,Clinton_pos,Sanders_neg,Sanders_neu,Sanders_pos,Rubio_neg,Rubio_neu,Rubio_pos,Cruz_neg,Cruz_neu,Cruz_pos
0,2006,8,31,kmikeym,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2006,12,22,so_n_so,0,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007,5,12,ursonate,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007,8,24,mtvmoonman,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2008,1,9,market_jp,NaN,NaN,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2008,1,10,rnz_news,NaN,NaN,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2008,2,21,dbrowell,NaN,NaN,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2008,3,4,dbrowell,NaN,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2008,3,6,barack,NaN,NaN,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2008,3,11,johnmiller,NaN,NaN,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# now read in our user file and merge that with the "supporters" information.

df_users = pd.read_csv('daily_users_supporters.csv')
df_users = df_users[['year','month','day','user','supports','frequency']]
df_users.head()

,year,month,day,user,supports,frequency
0,2006,3,21,rayreadyray,none,3
1,2006,3,30,adam,none,1
2,2006,3,30,dom,none,1
3,2006,3,30,noah,none,1
4,2006,3,30,rayreadyray,none,2


In [11]:
df_users[0:200]

,year,month,day,user,supports,frequency
0,2006,3,21,rayreadyray,none,3
1,2006,3,30,adam,none,1
2,2006,3,30,dom,none,1
3,2006,3,30,noah,none,1
4,2006,3,30,rayreadyray,none,2
5,2006,3,30,timroberts,none,2
6,2006,6,17,nic,none,1
7,2006,6,17,sharon,none,1
8,2006,6,17,telene,none,1
9,2006,8,31,kmikeym,none,1


In [12]:
df_merged = pd.merge(pivoted, df_users, on=['year','month','day','user'],how='outer')
df_merged.sort(['year','month','day','user'],inplace=True)
df_merged

,year,month,day,user,Trump_neg,Trump_neu,Trump_pos,Clinton_neg,Clinton_neu,Clinton_pos,...,Sanders_neu,Sanders_pos,Rubio_neg,Rubio_neu,Rubio_pos,Cruz_neg,Cruz_neu,Cruz_pos,supports,frequency
18232860,2006,3,21,rayreadyray,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,3
18232861,2006,3,30,adam,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,1
18232862,2006,3,30,dom,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,1
18232863,2006,3,30,noah,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,1
18232864,2006,3,30,rayreadyray,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,2
18232865,2006,3,30,timroberts,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,2
18232866,2006,6,17,nic,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,1
18232867,2006,6,17,sharon,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,1
18232868,2006,6,17,telene,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,1
0,2006,8,31,kmikeym,0,1,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,1


In [15]:
df_merged.fillna(0,inplace=True)
df_merged.head()

,year,month,day,user,Trump_neg,Trump_neu,Trump_pos,Clinton_neg,Clinton_neu,Clinton_pos,...,Sanders_neu,Sanders_pos,Rubio_neg,Rubio_neu,Rubio_pos,Cruz_neg,Cruz_neu,Cruz_pos,supports,frequency
18232860,2006,3,21,rayreadyray,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,none,3
18232861,2006,3,30,adam,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,none,1
18232862,2006,3,30,dom,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,none,1
18232863,2006,3,30,noah,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,none,1
18232864,2006,3,30,rayreadyray,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,none,2


In [16]:
df_merged.to_csv('daily_user_supporters_transformed.csv')